# Imports

Besides external libraries, we import several function from the local file : `utils.py`

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import brightway2 as bw

# Custom utils defined for inter-acv
from utils import *
from expression import *

# Init brightway2 and databases

In [ ]:
# Setup bw2
bw.projects.set_current('B_Publication')
bw.bw2setup()

# Import Ecoinvent DB (if not already done)
# Update the PATH to suit your installation
importDb(ECOINVENT_DB_NAME, './ecoinvent 3.4_cutoff_ecoSpold02/datasets')

# We use a separate DB for defining our model, reset it beforehand
resetDb(ACV_DB_NAME)

# Parameters are stored at project level : reset them
resetParams()

# Define parameters

We define the parameters of the model.

The numeric parametes are instances of sympy' 'Symbol' . Thus, any python arithmetic expression composed of parameters will not result into direct evaluation, but the creation of a symbolic formula that can be manipulated later.

In [ ]:
# Example 'float' parameter
a = newParamDef(
    'a', 
    ParamType.NUMBER, 
    default=0.5, 
    description="hello world")

b = newParamDef(
    'b', 
    ParamType.NUMBER, 
    default=0.5, 
    description="foo bar")

# Example 'enum' parameter, acting like a switch between several possibilities
# Enum parameters are not Symbol themselves
# They are a facility to represent many parameters at once '<paramName>_<enumValue>' 
# and should be used with the 'switch' method 
elec_switch_param = newParamDef(
    'elec_switch_param', 
    ParamType.ENUM, 
    values=["us", "eu"], 
    default="us", 
    description="Switch on electricty mix")

# Get references to background activities and products

`utils` provide two functions for easy and fast search of activities in reference databases : 
* **findBioAct** : Search activity in **biosphere3** db
* **findTechAct** : Search activity in **ecoinvent** db


In [ ]:
# Biosphere activities
ground_occupuation = findBioAct('Occupation, industrial area')
heat = findBioAct('Heat, waste', categories=['air'])

# Technosphere activities
alu = findTechAct("aluminium alloy production", "RER")

# Elec 
eu_elec = findTechAct("market group for electricity, medium voltage", 'ENTSO-E')
us_elec = findTechAct("market group for electricity, medium voltage", 'US')

# Define the model

The model is defined as a nested combination of reference activities and amount. 
Amounts are defined either as constant float values or algebric formulas implying the parameters defined above.

In [ ]:
# Create new, parametrized activity

# You can create a virtual "switch" activity combining an enum param and several other activities 
elec_switch = switch(elec_switch_param, {
    "us" : us_elec,
    "eu" : eu_elec})

# Create a new activity
activity1 = newActivity(
    name="act1", unit="kg", # Any extra named arg will be added as activity attribute
    exchanges= { # We define exhanges as a dict of 'activity : amount'
        ground_occupuation:3 + a, # Amount can be a fixed value 
        heat: b + 0.2,
        elec_switch:4}) # Amount can be a Sympy expression (any arithmetic expression of Parameters)

# The final model is just the root activity referencing the others
model = newActivity("model", "kg", {
        activity1 : b * 5 + a + 1, # Reference the activity we just created
        heat: 3,
        alu:0.4 * a}) 

In [ ]:
# Print_act displays activities as "pandas" tables
print_act(activity1) 
print_act(model)
          
# Note that symbolic expressions have not been evaluated at this stage

# Select the impacts to consider

In [ ]:
# List of impacts to consider
impacts = [m for m in bw.methods if 'ILCD 1.0.8 2016' in str(m) and 'no LT' in str(m)]

# Compute LCA

We provide two methods to compute LCA : 
* **multiLCA** : It uses brightway2 parametric capabilities. It is much slower and kept for comparing results.
* **multiLCAAlgebric** : It computes an algebric expression of the model and computes LCA once for all the background activities. Then it express each impact as a function of the parameter. This expression is compiled into 'numpy' native code, for fast computation on vectors of samples. This version is 1 million time faster.

In [ ]:
# Uses brightway2 parameters
multiLCA(us_elec, impacts, 
                   
    # Parameters of the model
    a=1, 
    b=2, 
    elec_switch_param="us")

In [ ]:
# Compute with algebric implementation : the values should be the same
multiLCAAlgebric(
    elec_switch, # The model 
    impacts, # Impacts
    
    # Parameters of the model
    a=1, 
    b=2,
    elec_switch_param="us")

In [ ]:
# Here is what the symbolic model looks like 
expr, _ = actToExpression(model)
expr

In [ ]:
# Fast computation for millions of separate samples
multiLCAWithParamsAlgebric(
    model, # The model 
    impacts, # Impacts
    
    # Parameters of the model
    alpha=list(range(1, 100000)), # LIst should be the same size for each param
    beta=list(range(1, 100000)),
    elec_switch_param="eu")